In [2]:
import json
from tensorflow.keras import optimizers
import numpy as np

data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('testing.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  

for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('down')

down


In [3]:
en_num_data = [[en2id[en] for en in line ] for line in endata]
ch_num_data = [[ch2id[ch] for ch in line] for line in chdata]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in chdata]

print('char:', endata[1])
print('index:', en_num_data[1])
print(len(en_num_data))
len(ch_num_data)
# de_num_data

char: Choose a recorder.
index: [22, 55, 62, 62, 9, 65, 10, 8, 10, 11, 65, 38, 62, 11, 39, 65, 11, 42]
100


100

In [4]:
import numpy as np

max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 49
max decoder length: 11
index data:
 [22, 55, 62, 62, 9, 65, 10, 8, 10, 11, 65, 38, 62, 11, 39, 65, 11, 42]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250


In [6]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [7]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [8]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

C:\Users\david\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 403)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  330752      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  675840      input_2[0][0]                    
                                                                 lstm[0][1]                   

5/5 [==============================] - 1s 182ms/step - loss: 0.5899 - accuracy: 0.6627
Epoch 61/250
5/5 [==============================] - 1s 161ms/step - loss: 0.5613 - accuracy: 0.6745
Epoch 62/250
5/5 [==============================] - 1s 153ms/step - loss: 0.5308 - accuracy: 0.6809
Epoch 63/250
5/5 [==============================] - 1s 153ms/step - loss: 0.5183 - accuracy: 0.6827
Epoch 64/250
5/5 [==============================] - 1s 156ms/step - loss: 0.5001 - accuracy: 0.6791
Epoch 65/250
5/5 [==============================] - 1s 154ms/step - loss: 0.4910 - accuracy: 0.6818
Epoch 66/250
5/5 [==============================] - 1s 156ms/step - loss: 0.4879 - accuracy: 0.6836
Epoch 67/250
5/5 [==============================] - 1s 154ms/step - loss: 0.4714 - accuracy: 0.6809
Epoch 68/250
5/5 [==============================] - 1s 158ms/step - loss: 0.4577 - accuracy: 0.6855
Epoch 69/250
5/5 [==============================] - 1s 164ms/step - loss: 0.4541 - accuracy: 0.6855
Epoch 70/250


5/5 [==============================] - 1s 173ms/step - loss: 1.5576 - accuracy: 0.4955
Epoch 142/250
5/5 [==============================] - 1s 155ms/step - loss: 1.4322 - accuracy: 0.5082
Epoch 143/250
5/5 [==============================] - 1s 166ms/step - loss: 1.3618 - accuracy: 0.5155
Epoch 144/250
5/5 [==============================] - 1s 168ms/step - loss: 1.2836 - accuracy: 0.5191
Epoch 145/250
5/5 [==============================] - 1s 172ms/step - loss: 1.2213 - accuracy: 0.5336
Epoch 146/250
5/5 [==============================] - 1s 162ms/step - loss: 1.2261 - accuracy: 0.5427
Epoch 147/250
5/5 [==============================] - 1s 162ms/step - loss: 1.1559 - accuracy: 0.5455
Epoch 148/250
5/5 [==============================] - 1s 153ms/step - loss: 1.0918 - accuracy: 0.5509
Epoch 149/250
5/5 [==============================] - 1s 153ms/step - loss: 1.0336 - accuracy: 0.5500
Epoch 150/250
5/5 [==============================] - 1s 155ms/step - loss: 0.9944 - accuracy: 0.5664
Epoc

5/5 [==============================] - 1s 157ms/step - loss: 0.3396 - accuracy: 0.7009
Epoch 223/250
5/5 [==============================] - 1s 156ms/step - loss: 0.3422 - accuracy: 0.7009
Epoch 224/250
5/5 [==============================] - 1s 155ms/step - loss: 0.3418 - accuracy: 0.7000
Epoch 225/250
5/5 [==============================] - 1s 156ms/step - loss: 0.3525 - accuracy: 0.6991
Epoch 226/250
5/5 [==============================] - 1s 154ms/step - loss: 0.3497 - accuracy: 0.6991
Epoch 227/250
5/5 [==============================] - 1s 154ms/step - loss: 0.3417 - accuracy: 0.6991
Epoch 228/250
5/5 [==============================] - 1s 154ms/step - loss: 0.3394 - accuracy: 0.6991
Epoch 229/250
5/5 [==============================] - 1s 162ms/step - loss: 0.3346 - accuracy: 0.7018
Epoch 230/250
5/5 [==============================] - 1s 154ms/step - loss: 0.3349 - accuracy: 0.7055
Epoch 231/250
5/5 [==============================] - 1s 154ms/step - loss: 0.3275 - accuracy: 0.7036
Epoc